In [194]:
from nltk.tokenize import sent_tokenize, RegexpTokenizer
import os
import re
import sqlite3
import spacy
from tqdm import tqdm

# Предобработка

Размер корпуса - более 2 миллионов словоупотреблений.

В качестве источника текстов выбрала сайт [lib.ru](http://www.lib.ru), который позволяет скачать txt-версии необходимых произведений. Удаление нетекстовых элементов происходило вручную (поскольку краулером это сделать было просто невозможно, к тому же, если сайт предоставляет возможность получить txt-файлы, кажется резонным воспользоваться такой возможностью):
1. Удалялись пометы об авторской орфографии и т.п.
2. В некоторых файлах символ "J" заменялся на "ё"
3. Удалялась жанровая принадлежность
4. Удалялись номера страниц в фигурных скобках

Сначала был создан словарь соответствий произведений ссылкам (ключ - название txt-файла, значение - ссылка на произведение).

In [195]:
url_for_file = {
    "ivandenisych.txt": "http://www.lib.ru/PROZA/SOLZHENICYN/ivandenisych.txt",
    "matren.txt": "http://www.lib.ru/PROZA/SOLZHENICYN/matren.txt",
    "na_izlomah.txt": "http://www.lib.ru/PROZA/SOLZHENICYN/na_izlomah.txt",
    "r_dvarasskaza.txt": "http://www.lib.ru/PROZA/SOLZHENICYN/r_dvarasskaza.txt",
    "r_hod.txt": "http://www.lib.ru/PROZA/SOLZHENICYN/r_hod.txt",
    "r_kochet.txt": "http://www.lib.ru/PROZA/SOLZHENICYN/r_kochet.txt",
    "rk.txt": "http://www.lib.ru/PROZA/SOLZHENICYN/rk.txt",
    "sol_gv.txt": "http://www.lib.ru/PROZA/SOLZHENICYN/sol_gv.txt",
    "solg_as.txt": "http://www.lib.ru/PROZA/SOLZHENICYN/solg_as.txt",
    "vkp1.txt": "http://www.lib.ru/PROZA/SOLZHENICYN/vkp1.txt",
    "vkp2.txt": "http://www.lib.ru/PROZA/SOLZHENICYN/vkp2.txt"
}

Далее собираю список словарей, содержащих информацию о произведении (автор, название, текст и ссылка, взятая по названию файла из словаря в предыдущей ячейке).

In [196]:
all_info = []

for book in os.listdir(r"project"):
    with open(
        os.path.join(r"project", book), "r", encoding="ANSI"
    ) as f:
        text_info = {}
        file = f.read()
        text_info["author"] = file.split("\n")[0].split(". ")[0]
        text_info["name"] = file.split("\n")[0].split(". ")[1]
        text_info["text"] = sent_tokenize("\n".join(file.split("\n")[1:]))
        text_info["source"] = url_for_file[book]
        all_info.append(text_info)

# Морфологический анализ

Изначально хотела использовать в качестве парсера stanza, но в библиотеке не оказалось русских лемм, поэтому пришлось поменять на spacy.

In [197]:
nlp = spacy.load("ru_core_news_md")

Функция токенизации необходима для того, чтобы запрос выполнялся без учета пунктуации (вряд ли кто-либо будет искать n-граммы, включая в запрос запятые, да и пунктуационная разметка не входила в задачи написания корпуса). Токенизироваться будет каждое предложение при записи в ту таблицу базы данных, где будут храниться словоформы, их леммы и грамматические теги.

In [198]:
def tokenization(text):
    """
    Функция для токенизации текстов
    Аргумент - строка с текстом отзыва
    Вывод - список токенов
    """
    text = text.lower()
    tokenizer = RegexpTokenizer(r"[а-я0-9ё]+")
    text_tokenized = tokenizer.tokenize(text)
    return " ".join(text_tokenized)

Функция записи в базу данных будет заполнять сразу все три таблицы базы.

In [210]:
def write_to_db(block):
    """
    Функция для записи информации о произведении в базу данных
    Аргумент - блок-словарь с мета-информацией и текстом произведения
    """
    cur.execute(
        """
        INSERT INTO meta_info 
            (book_title, author, source) VALUES (?, ?, ?)
            """,
            (
            block["name"],
            block["author"],
            block["source"]
            )
    )
    conn.commit()
    
    meta = cur.execute("""SELECT meta_id FROM meta_info""").fetchone()[0]
    
    for sentence in block["text"]:
        cur.execute(
            """
            INSERT INTO sentences
                (meta_id, sentence) VALUES (?, ?)
            """,
            (
                meta,
                sentence
            ),
        )
        conn.commit()
        
    cur.execute("SELECT sentence_id, sentence FROM sentences")
    
    for sent_id, sent_text in cur.fetchall():
        doc = nlp(tokenization(sent_text))
        spacy_parse = [
            tuple([str(token.text), str(token.lemma_), str(token.pos_)])
            for token in doc
        ]
        
        for parse in spacy_parse:        
            cur.execute(
                    """
                    INSERT INTO parsing
                    (sentence_id, word, lemma, pos) VALUES (?, ?, ?, ?)
                    """,
                    (
                        sent_id,
                        parse[0],
                        parse[1],
                        parse[2]
                    ),
                )
            conn.commit()

Устанавливаем соединение с базой данных.

In [200]:
conn = sqlite3.connect("solgenitsin.db")
cur = conn.cursor()

Создаем три таблицы:

1. Мета-информация о произведении (название, автор, источник-ссылка)
2. Предложения (айди произведения, из которого взято предложение, и само предложение)
3. Морфологическая информация (айди предложения, из которого взято слово, само слово, лемма, грамматический тег)

In [202]:
cur.execute(
    """
CREATE TABLE IF NOT EXISTS meta_info (
    meta_id INTEGER PRIMARY KEY AUTOINCREMENT,
    book_title TEXT,
    author TEXT,
    source TEXT
)
"""
)

cur.execute(
    """
CREATE TABLE IF NOT EXISTS sentences (
    sentence_id INTEGER PRIMARY KEY AUTOINCREMENT,
    meta_id INTEGER,
    sentence TEXT
)
"""
)

cur.execute(
    """
CREATE TABLE IF NOT EXISTS parsing (
    word_id INTEGER PRIMARY KEY AUTOINCREMENT,
    sentence_id INTEGER,
    word TEXT,
    lemma TEXT,
    pos TEXT
)
"""
)

conn.commit()

Дальше проходимся по списку словарей с информацией по произведениям, составленному ранее, и мучительно долго записываем все в базу данных.

In [203]:
for item in tqdm(all_info):
    write_to_db(item)

100%|██████████████████████████████████████████████████████████████████████████████| 11/11 [7:24:21<00:00, 2423.80s/it]


# Функция поиска

Сначала необходимо выделить все грамматические теги, которые встречаются в разметке (для дальнейшего использования в функции поиска).

In [204]:
tags_query = """SELECT pos FROM parsing"""
cur.execute(tags_query)
print(set([tag[0] for tag in cur.fetchall()]))

{'PRON', 'ADP', 'VERB', 'AUX', 'CCONJ', 'DET', 'PUNCT', 'NOUN', 'NUM', 'X', 'ADV', 'SYM', 'PROPN', 'PART', 'SCONJ', 'INTJ', 'ADJ'}


Далее идет функция поиска, которая получает на вход запрос, делит его по пробелам. Проходясь по каждому компоненту запроса, функция делает соответствующий запрос в базу данных и записывает в общий список соответствий списки соответствий по каждому компоненту (в списках находятся кортежи с айди предложения и айди совпавшего слова). Далее происходит уточнение, n-граммы какого типа мы ищем. Наконец, в случае биграмм и триграмм идет перебор всех кортежей в первом списке (т.е. соответствий первому компоненту запроса) с проверкой наличия элементов на расстоянии 1 (и в случае триграмм и их последнего елемента, на расстоянии 2) в предложении с таким же айди, то есть проверяется есть ли в следующих списках кортежи, в которых айди слова отличается на 1 (или 2) от айди первого слова, а айди предложения совпадают. Айди подходящих предложений записываются в отдельный список. По этим айди происходит получение необходимой мета-информации, которую функция возвращает в виде списка кортежей. 

In [205]:
def search(query):
    """
    Функция для поиска соответствий заданному запросу
    Аргумент - запрос
    Вывод - список кортежей с текстом и мета-информацией каждого подходящего предложения
    """
    queries_dict = {
        'wordform': """SELECT word_id, sentence_id FROM parsing WHERE word=?""",
        'lemma_and_pos': """SELECT word_id, sentence_id FROM parsing WHERE lemma=? AND pos=?""",
        'pos': """SELECT word_id, sentence_id FROM parsing WHERE pos=?""",
        'lemma': """SELECT word_id, sentence_id FROM parsing WHERE lemma=?"""
    }
        
    tags = {'PRON', 'ADP', 'VERB', 'AUX', 'CCONJ', 'DET', 'PUNCT', 'NOUN', 'NUM', 'X', 'ADV', 'SYM', 'PROPN', 'PART', 'SCONJ', 'INTJ', 'ADJ'}
    
    query_res = [] # список, в который помещаются списки соответствий каждому компоненту запроса
    
    ngramm = query.split(" ")

    for element in ngramm:
        if re.match(r'".+"', element) is not None:
            element = element[1:-1]
            cur.execute(queries_dict['wordform'], (element, ))
        elif "+" in element:
            cur.execute(queries_dict['lemma_and_pos'], tuple(element.split("+")))
        elif element in tags:
            cur.execute(queries_dict['pos'], (element, ))
        else:
            doc = nlp(element)
            lemma = [token.lemma_ for token in doc][0]
            cur.execute(queries_dict['lemma'], (lemma, ))
        query_res.append(cur.fetchall())
    
    golden = [] # список айди подходящих предложений
    
    if len(query_res) == len(ngramm): # если найдены соответствия всем компонентам n-граммы
        if len(ngramm) == 1: # если поиск униграммы
            for res in query_res:
                for item in res:
                    golden.append(item[1])
        elif len(ngramm) == 2: # если поиск биграммы
            for item in query_res[0]:
                if tuple([item[0] + 1, item[1]]) in query_res[1]:
                    golden.append(item[1])
        elif len(ngramm) == 3: # если поиск триграммы
            for item in query_res[0]:
                if tuple([item[0] + 1, item[1]]) in query_res[1] and tuple([item[0] + 2, item[1]]) in query_res[2]:
                    golden.append(item[1])
                
    golden_tuples = []
                
    for golden_id in golden:
        cur.execute("""SELECT sentence, book_title, author, source FROM sentences JOIN meta_info ON sentences.meta_id = meta_info.meta_id WHERE sentence_id=?""", (golden_id, ))
        golden_tuples.append(cur.fetchone())
        
    return golden_tuples

# Тестирование

На данный момент выводы ячеек очищены от предыдущих тестов. Поскольку объем корпуса достаточно большой, запись осуществляется в генерируемый файл (и происходит достаточно долго).

**Примеры запросов:**

Униграммы:
1. NUM
2. было+VERB (либо, напротив, следующий запрос: было+AUX)
3. "ступая"

Биграммы:
1. без NOUN
2. рассматривающий+ADJ NOUN
3. далее+ADV VERB

Триграммы:
1. NOUN и NOUN
2. PRON VERB на
3. не VERB NOUN

In [ ]:
req = '' # сюда добавить запрос

In [ ]:
results = search(req)

if len(results) == 0:
    print('К сожалению, по Вашему запросу ничего не найдено. Проверьте, что Вы не забыли про букву "ё", или попробуйте ввести другой запрос.')
else:
    with open("results.txt", "w", encoding="utf_8") as f:
        for item in results:
            f.write(f'{item[0]}\nИсточник: {item[2]} "{item[1]}" (доступно по ссылке: {item[3]} )\n---\n')
    print("По Вашему запросу был создан файл с выдачей.")